# Autómatas finitos y expresiones regulares

In [1]:
class Puerta:
    def __init__(self):
        self._estado = 'Cerrada'
    
    def abrir(self):
        self._estado = 'Abierta'
    
    def cerrar(self):
        self._estado = 'Cerrada'
    
    @property
    def estado(self):
        return self._estado

In [2]:
obj = Puerta()
obj.estado

'Cerrada'

In [3]:
obj.abrir()
obj.estado

'Abierta'

In [4]:
obj.cerrar()
obj.estado

'Cerrada'

In [5]:
obj.cerrar()
obj.estado

'Cerrada'

**EJEMPLO**
Se desea cruzar un costal de maiz, una gallina y un perro al otro lado del río.
La canoa sólo tiene espacio para ti y para un otro pasajero, pero la gallina no
puede quedarse sola con el costal de maíz porque se lo come, ni tampoco puede
quedarse sola con el perro porque el perro 

In [6]:
class AcertijoRio:
    def __init__(self):
        self.costal = False
        self.gallina = False
        self.perro = False
        self.persona = False
    
    @property
    def estado_invalido(self):
        return (self.gallina == self.costal != self.persona
                or self.perro == self.gallina != self.persona)
    
    def cruzar_costal(self):
        if self.estado_invalido: return
        if self.persona != self.costal: return
        self.persona = self.costal = not self.persona
    
    def cruzar_gallina(self):
        if self.estado_invalido: return
        if self.persona != self.gallina: return
        self.persona = self.gallina = not self.persona
    
    def cruzar_perro(self):
        if self.estado_invalido: return
        if self.persona != self.perro: return
        self.persona = self.perro = not self.persona
    
    def cruzar_persona(self):
        if self.estado_invalido: return
        self.persona = not self.persona
    
    def __repr__(self):
        letras = 'CGPH'
        estados = [self.costal, self.gallina, self.perro, self.persona]
        izquierda = [letra for (letra, estado) in zip(letras, estados)
                     if not estado]
        derecha = [letra for (letra, estado) in zip(letras, estados) if estado]
        return f"{''.join(izquierda)}-{''.join(derecha)}"

In [7]:
acertijo = AcertijoRio()
acertijo

CGPH-

In [8]:
acertijo.estado_invalido

False

In [9]:
acertijo.cruzar_gallina()
acertijo

CP-GH

In [10]:
acertijo.cruzar_persona()
acertijo

CPH-G

In [11]:
acertijo.cruzar_costal()
acertijo

P-CGH

In [12]:
acertijo.cruzar_persona()
acertijo

PH-CG

In [13]:
acertijo.estado_invalido

True

In [14]:
acertijo.cruzar_persona()
acertijo

PH-CG

## Implementación de un intérprete de autómatas finitos en general

Continuando el ejemplo anterior, denotamos:
- `C`: costal de maíz
- `G`: gallina
- `P`: perro
- `H`: persona
- `-`: el río

In [15]:
# Función de transición
δ = {
    'CGPH-': {'C': 'GP-CH', 'G': 'CP-GH', 'P': 'CG-PH', 'H': 'CGP-H'},
    'GP-CH': {'C': 'GP-CH', 'G': 'GP-CH', 'P': 'GP-CH', 'H': 'GP-CH'},
    'CP-GH': {'C': 'CP-GH', 'G': 'CGPH-', 'P': 'CP-GH', 'H': 'CPH-G'},
    'CG-PH': {'C': 'CG-PH', 'G': 'CG-PH', 'P': 'CG-PH', 'H': 'CG-PH'},
    'CGP-H': {'C': 'CGP-H', 'G': 'CGP-H', 'P': 'CGP-H', 'H': 'CGP-H'},
    'CPH-G': {'C': 'P-CGH', 'G': 'CPH-G', 'P': 'C-GPH', 'H': 'CP-GH'},
    'P-CGH': {'C': 'CPH-G', 'G': 'GPH-C', 'P': 'P-CGH', 'H': 'PH-CG'},
    'C-GPH': {'C': 'C-GPH', 'G': 'CGH-P', 'P': 'CPH-G', 'H': 'CH-GP'},
    'GPH-C': {'C': 'GPH-C', 'G': 'P-CGH', 'P': 'G-CPH', 'H': 'GP-CH'},
    'PH-CG': {'C': 'PH-CG', 'G': 'PH-CG', 'P': 'PH-CG', 'H': 'PH-CG'},
    'CGH-P': {'C': 'G-CPH', 'G': 'C-GPH', 'P': 'CGH-P', 'H': 'CG-PH'},
    'CH-GP': {'C': 'CH-GP', 'G': 'CH-GP', 'P': 'CH-GP', 'H': 'CH-GP'},
    'G-CPH': {'C': 'CGH-P', 'G': 'G-CPH', 'P': 'GPH-C', 'H': 'GH-CP'},
    'GH-CP': {'C': 'GH-CP', 'G': '-CGPH', 'P': 'GH-CP', 'H': 'G-CPH'},
    '-CGPH': {'C': 'CH-GP', 'G': 'GH-CP', 'P': 'PH-CG', 'H': 'H-CGP'},
    'H-CGP': {'C': 'H-CGP', 'G': 'H-CGP', 'P': 'H-CGP', 'H': 'H-CGP'},
}

# Estado inicial
q0 = 'CGPH-'

# Estados de aceptación
F = ['-CGPH']

# Definición del autómata
M = (δ, q0, F)

In [16]:
def ejecutar(M, w):
    (δ, q0, F) = M
    q = q0
    for s in w:
        q = δ[q][s]
    return q in F

In [17]:
ejecutar(M, 'GHCGPHG')

True

In [18]:
ejecutar(M, 'GHPGCHG')

True

In [19]:
ejecutar(M, 'GPHHP')

False

In [20]:
w = input('Cadena de entrada: ')
if ejecutar(M, w):
    print('Cadena aceptada.')
else: print('Cadena rechazada.')

Cadena de entrada:  GH


Cadena rechazada.


## Definición formal de un Autómata Finito (Determinista)

Un **autómata finito** es una 5-ada $(Q, \Sigma, \delta, q_0, F)$
- $Q$ es un conjunto finito llamado conjunto de **estados**.
- $\Sigma$ es un conjunto finito llamado **alfabeto**.
- $\delta: Q \times \Sigma \to Q$ es la **función de transición**.
- $q_0 \in Q$ es el **estado inicial**.
- $F \subseteq Q$ es el conjunto de **estados de aceptación**.

**Ejemplo** (tal como se vio en el WhiteBoard de Office):
$M_1 = (Q, \Sigma, q_1, F)$
- $Q = \lbrace q_1, q_2, q_3 \rbrace$
- $\Sigma = \{ 0, 1 \}$
- $\delta$ está dada por
| $\delta$ | 0     | 1     |
|----------|-------|-------|
| $q_1$    | $q_1$ | $q_2$ |
| $q_2$    | $q_3$ | $q_2$ |
| $q_3$    | $q_2$ | $q_2$ |
- $q_1$ es el estado inicial
- $F = \{ q_2 \}$

**Definición** Si $M$ es un AFD, el **lenguaje de la máquina $M$**, denotado 
por $L(M)$ es el conjunto de todas las cadenas $w$ sobre $\Sigma$ que acepta el
autómata $M$, es decir que dada la cadena 
$w = w_1w_2w_3\cdots w_n \in \Sigma^\star$ existe una sucesión finita de
estados $r_0, r_1, r_2, \dots, r_n$ en $Q$ que cumple las siguientes
condiciones:
1. $r_0 = q_0$
2. $\delta(r_i, w_{i + 1}) = r_{i + 1}$
3. $r_n \in F$

**Ejemplo** Una máquina $M_2$ cuyo lenguaje son las cadenas que tienen un par
de unos:
$M_2 = (Q, \Sigma, \mathit{par}, F)$
- $Q = \{ \mathit{par}, \mathit{impar} \}$
- $\Sigma = \{0, 1\}$
- $\delta$ está dada por
| $\delta$         | 0                | 1                |
|------------------|------------------|------------------|
| $\mathit{par}$   | $\mathit{par}$   | $\mathit{impar}$ |
| $\mathit{impar}$ | $\mathit{impar}$ | $\mathit{par}$   |
- $\mathit{par}$ es el estado inicial
- $F = \{ \mathit{par} \}$

**Ejemplo** Una máquina $M_3$ cuyo lenguaje son las cadenas que tienen un par
de unos y de ceros:
$M_3 = (Q, \Sigma, \mathit{PP}, F)$
- $Q = \{ \mathit{PP}, \mathit{PI}, \mathit{IP}, \mathit{II} \}$
- $\Sigma = \{0, 1\}$
- $\delta$ está dada por
| $\delta$      | 0             | 1             |
|---------------|---------------|---------------|
| $\mathit{PP}$ | $\mathit{IP}$ | $\mathit{PI}$ |
| $\mathit{IP}$ | $\mathit{PP}$ | $\mathit{II}$ |
| $\mathit{PI}$ | $\mathit{II}$ | $\mathit{PP}$ |
| $\mathit{II}$ | $\mathit{PI}$ | $\mathit{IP}$ |
- $\mathit{PP}$ es el estado inicial
- $F = \{ \mathit{PP} \}$

**Ejemplo** Una máquina $B_m$ que acepta las cadenas con un múltiplo de $m$
bits 1.
- $Q = \{q_0, q_1, q_2, \ldots, q_m\}$
- $\Sigma = \{0, 1\}$
- $\delta$ está dada por
  - $\delta(q_{m - 1}, 1) = q_0$
  - $\delta(q_j, 0) = q_j$
  - $\delta(q_j, 1) = q_{j + 1}$
- $q_0$ es el estado inicial
- $F = \{ q_0 \}$

### Uso de la implementación

Un autómata finito determinista se puede representar mediante un **programa**
que consiste en un conjunto finito de tripletas $(q, s, r) \in Q \times \Sigma
\times Q$.
El programa se puede convertir a una función de transición asociando la
tripleta $(q, s, r)$ con $\delta(q, s) = r$.
Note que el programa para poder definir una función de trancisión requiere dos
condiciones:
1. No pueden existir dos o más tripletas $(q, s, r)$ y $(q^\prime, s^\prime, t)$
   con $(q, s) = (q^\prime, s^\prime)$ porque $\delta$ entonces ya no es una
   función.
2. Para cada pareja $(q, s) \in Q \times \Sigma$ existe una y exactamente una
   tripleta $(q^\prime, s^\prime, r)$ con $(q, s) = (q^\prime, s^\prime)$

In [21]:
from curso import automatasfinitos as af

In [22]:
programa = [
    af.Transicion("q1", "0", "q1"),
    af.Transicion("q1", "1", "q2"),
    af.Transicion("q2", "0", "q3"),
    af.Transicion("q2", "1", "q2"),
    af.Transicion("q3", "0", "q2"),
    af.Transicion("q3", "1", "q2"),
]
M = af.AFD(programa, 'q1', ['q2'])

In [23]:
M('00100011101')

True

In [24]:
M('00100011100')

True

In [25]:
M('0010001110000')

True

In [26]:
M('00111010000')

True

## Lenguajes regulares y sus operaciones

**Definición** Un lenguaje es **regular** si es reconocido por un autómata
finito. Dados $A$ y $B$ lenguajes regulares, las operaciones regularesson
estas:
- $A \cup B = \{x | x \in A \text{ o } x \in B\}$
- $A \circ B = \{xy | x \in A \text{ y } y \in B\}$
- $A^\star = \{x_1x_2\cdots x_k | \text{cada } x_i \in A \text{ para cualquier 
  } k \ge 0\}$

**Ejemplos**
Sean $A = \{\texttt{cara}, \texttt{cola}\}$,
$B = \{\texttt{col}, \texttt{chones}\}$.
- $A \cup B = \{\texttt{cara}, \texttt{cola}, \texttt{col}, \texttt{chones}\}$.
- $A \circ B = \{ \texttt{caracol}, \texttt{carachones}, \texttt{colacol},
  \texttt{colachones}\}$
- $A^\star$ contiene elementos como $\epsilon$, $\texttt{cara}$,
  $\texttt{cola}$, $\texttt{caracola}$, $\texttt{colacara}$,
  $\texttt{caracara}$, $\texttt{colacola}$, $\texttt{caracaracara}$,
  $\texttt{caracolacara}$,...

#### Autómatas finitos no deterministas (AFN)

En un autómata finito no determinista permitimos que el programa no defina una
función, sino una relación arbitraria $Q \times \Sigma_\epsilon \times Q$ donde
$\Sigma_\epsilon = \Sigma \cup \{\epsilon\}$.

Esto nos permitirá hacer un programa no determinista donde en cada estado:
1. Podemos pasar a algún otro estado sin consumir ningún símbolo de entrada
   mediante una transición $\epsilon$.
2. Podemos consumir un símbolo de la entrada para pasar a cero, uno o más
   estados siguientes.

**Definición** Un autómata finito no determinista es una 5-ada $(Q, \Sigma,
q_0, F)$ donde
1. $Q$ es un conjunto finito de *estados*
2. $\Sigma$ es un *alfabeto* finito
3. $\delta: Q\times \Sigma_{\epsilon} \to \mathcal{P}(Q)$ es una función
   parcial *de trancisión*.
4. $q_0$ el *estado inicial*.
5. $F \subseteq Q$ es el conjunto de *estados de aceptación*.

**Definición** Si $N$ es un AFN, el **lenguaje de la máquina $N$**, denotado 
por $L(N)$ es el conjunto de todas las cadenas $w$ sobre $\Sigma$ que acepta el
autómata $N$, es decir que dada la cadena 
$w = y_1y_2y_3\cdots y_m \in \Sigma_{\epsilon}^\star$ existe una sucesión
finita de estados $r_0, r_1, r_2, \dots, r_m$ en $Q$ que cumple las siguientes
condiciones:
1. $r_0 = q_0$
2. $r_{i + 1} \in \delta(r_i, y_{i + 1})$
3. $r_m \in F$

**Teorema** Sea $L$ es un lenguaje reconocido por un AFN, entonces existe un
AFD que lo reconoce.

In [27]:
import collections

def hacer_determinista(N: af.AFN) -> af.AFD:
    """
    Crea un AFD a partir del autómata N que reconoce el mismo lenguaje.
    """
    # Calcular metaestado inicial
    metaestado_inicial = frozenset(N.cerradura_epsilon([N.estado_inicial]))

    # Calcular el metaprograma
    metaestados = {metaestado_inicial}  # Metaestados vistos
    metaprograma = set()
    cola = collections.deque([metaestado_inicial])  # Metaestados por explorar
    while cola:
        metaestado = cola.popleft()
        for simbolo in N.alfabeto:
            # Descubrir los metaestados accesibles
            siguientes = {siguiente
                          for estado in metaestado
                          for siguiente in N.transicion(estado, simbolo)}
            siguientes.update(N.cerradura_epsilon(siguientes))
            metaestado_siguiente = frozenset(siguientes)
            metaprograma.add(
                af.Transicion(metaestado, simbolo, metaestado_siguiente))
            if metaestado_siguiente not in metaestados:
                metaestados.add(metaestado_siguiente)
                cola.append(metaestado_siguiente)
    
    # Calcular metaestados finales
    metaestados_finales = {metaestado
                           for metaestado in metaestados
                           if any(N.es_de_aceptacion(estado)
                                  for estado in metaestado)}
    
    return af.AFD(metaprograma, metaestado_inicial, metaestados_finales)

In [28]:
programa = [
    af.Transicion('q1', '0', 'q1'),
    af.Transicion('q1', '1', 'q1'),
    af.Transicion('q1', '1', 'q2'),
    af.Transicion('q2', '0', 'q3'),
    af.Transicion('q2', '',  'q3'),
    af.Transicion('q3', '1', 'q4'),
    af.Transicion('q4', '0', 'q4'),
    af.Transicion('q4', '1', 'q4'),
]
estado_inicial = 'q1'
estados_de_aceptacion = ['q4']
N = af.AFN(programa, estado_inicial, estados_de_aceptacion)

In [29]:
N('0110110')

True

In [30]:
N.programa

(Transicion(estado='q1', simbolo='0', siguiente='q1'),
 Transicion(estado='q1', simbolo='1', siguiente='q1'),
 Transicion(estado='q1', simbolo='1', siguiente='q2'),
 Transicion(estado='q2', simbolo='0', siguiente='q3'),
 Transicion(estado='q2', simbolo='', siguiente='q3'),
 Transicion(estado='q3', simbolo='1', siguiente='q4'),
 Transicion(estado='q4', simbolo='0', siguiente='q4'),
 Transicion(estado='q4', simbolo='1', siguiente='q4'))

In [31]:
M = hacer_determinista(N)
M.programa

(Transicion(estado=frozenset({'q1', 'q3'}), simbolo='1', siguiente=frozenset({'q1', 'q3', 'q2', 'q4'})),
 Transicion(estado=frozenset({'q1', 'q3', 'q2'}), simbolo='0', siguiente=frozenset({'q1', 'q3'})),
 Transicion(estado=frozenset({'q1'}), simbolo='1', siguiente=frozenset({'q1', 'q3', 'q2'})),
 Transicion(estado=frozenset({'q1', 'q4'}), simbolo='0', siguiente=frozenset({'q1', 'q4'})),
 Transicion(estado=frozenset({'q1', 'q3', 'q4'}), simbolo='1', siguiente=frozenset({'q1', 'q3', 'q2', 'q4'})),
 Transicion(estado=frozenset({'q1', 'q3'}), simbolo='0', siguiente=frozenset({'q1'})),
 Transicion(estado=frozenset({'q1'}), simbolo='0', siguiente=frozenset({'q1'})),
 Transicion(estado=frozenset({'q1', 'q3', 'q2'}), simbolo='1', siguiente=frozenset({'q1', 'q3', 'q2', 'q4'})),
 Transicion(estado=frozenset({'q1', 'q3', 'q2', 'q4'}), simbolo='0', siguiente=frozenset({'q1', 'q3', 'q4'})),
 Transicion(estado=frozenset({'q1', 'q3', 'q4'}), simbolo='0', siguiente=frozenset({'q1', 'q4'})),
 Transicio

In [32]:
def estados_enteros(automata: af.AutomataFinito, ordenar=True, inicio=0) \
        -> af.AutomataFinito:
    estados = automata.estados
    if ordenar:
        estados = sorted(estados)
    indice = {estado: numero
              for numero, estado in enumerate(estados, inicio)}
    programa = [af.Transicion(indice[estado], simbolo, indice[siguiente])
                for estado, simbolo, siguiente in automata.programa]
    estado_inicial = indice[automata.estado_inicial]
    estados_de_aceptacion = [indice[estado]
                             for estado in automata.estados_de_aceptacion]
    return type(automata)(programa, estado_inicial, estados_de_aceptacion)

In [33]:
M = estados_enteros(M, inicio=10)
M.programa

(Transicion(estado=14, simbolo='1', siguiente=15),
 Transicion(estado=12, simbolo='0', siguiente=14),
 Transicion(estado=10, simbolo='1', siguiente=12),
 Transicion(estado=13, simbolo='0', siguiente=13),
 Transicion(estado=11, simbolo='1', siguiente=15),
 Transicion(estado=14, simbolo='0', siguiente=10),
 Transicion(estado=10, simbolo='0', siguiente=10),
 Transicion(estado=12, simbolo='1', siguiente=15),
 Transicion(estado=15, simbolo='0', siguiente=11),
 Transicion(estado=11, simbolo='0', siguiente=13),
 Transicion(estado=15, simbolo='1', siguiente=15),
 Transicion(estado=13, simbolo='1', siguiente=15))

In [34]:
def concatenacion(NA: af.AFN, NB: af.AFN) -> af.AFN:
    NA = estados_enteros(NA)
    NB = estados_enteros(NB, inicio=len(NA.estados))
    
    programa = list(NA.programa) + list(NB.programa)
    for estado, simbolo, siguiente in NA.programa:
        if siguiente not in NA.estados_de_aceptacion: continue
        programa.append(af.Transicion(siguiente, '', NB.estado_inicial))
    
    estado_inicial = NA.estado_inicial
    
    estados_de_aceptacion = NB.estados_de_aceptacion
    
    return af.AFN(programa, estado_inicial, estados_de_aceptacion)

Ejemplo:
Sean
$A = \{w | w \text{ tiene un número par de ceros}\}$
y
$B = \{w | w \text{ termina en 00100}\}$.

In [35]:
NA = af.AFN(
    programa=[
        af.Transicion(0, '0', 1),
        af.Transicion(0, '1', 0),
        af.Transicion(1, '1', 1),
        af.Transicion(1, '0', 0),
    ],
    estado_inicial=0,
    estados_de_aceptacion=[1]
)

NB = af.AFN(
    programa=[
        af.Transicion(2, '0', 2),
        af.Transicion(2, '1', 2),
        af.Transicion(2, '0', 3),
        af.Transicion(3, '0', 4),
        af.Transicion(4, '1', 5),
        af.Transicion(5, '0', 6),
        af.Transicion(6, '0', 7),
    ],
    estado_inicial=2,
    estados_de_aceptacion=[7]
)

In [36]:
NA('00111010')

False

In [37]:
NA('10010')

True

In [38]:
NB('00000100')

True

In [39]:
NC = concatenacion(NA, NB)
NC.programa

(Transicion(estado=0, simbolo='0', siguiente=1),
 Transicion(estado=0, simbolo='1', siguiente=0),
 Transicion(estado=1, simbolo='1', siguiente=1),
 Transicion(estado=1, simbolo='0', siguiente=0),
 Transicion(estado=1, simbolo='', siguiente=2),
 Transicion(estado=2, simbolo='0', siguiente=2),
 Transicion(estado=2, simbolo='0', siguiente=3),
 Transicion(estado=2, simbolo='1', siguiente=2),
 Transicion(estado=3, simbolo='0', siguiente=4),
 Transicion(estado=4, simbolo='1', siguiente=5),
 Transicion(estado=5, simbolo='0', siguiente=6),
 Transicion(estado=6, simbolo='0', siguiente=7))

In [40]:
NC.estado_inicial

0

In [41]:
NC.estados_de_aceptacion

frozenset({7})

In [42]:
NC('010100010100100')

True

In [43]:
NC('11100100')

False

In [45]:
def union(NA: af.AFN, NB: af.AFN) -> af.AFN:
    """
    Combina los autómatas NA y NB en un autómata cuyo lenguaje es la
    unión de los lenguajes de NA y NB.
    """
    NA = estados_enteros(NA, inicio=1)
    NB = estados_enteros(NB, inicio=len(NA.estados) + 1)
    
    programa = list(NA.programa) + list(NB.programa)
    programa.append(af.Transicion(0, '', NA.estado_inicial))
    programa.append(af.Transicion(0, '', NB.estado_inicial))
    
    estado_inicial=0
    estados_de_aceptacion = list(NA.estados_de_aceptacion) \
                            + list(NB.estados_de_aceptacion)
    
    return af.AFN(programa, estado_inicial, estados_de_aceptacion)

In [47]:
NC = union(NA, NB)
NC.programa

(Transicion(estado=1, simbolo='0', siguiente=2),
 Transicion(estado=1, simbolo='1', siguiente=1),
 Transicion(estado=2, simbolo='1', siguiente=2),
 Transicion(estado=2, simbolo='0', siguiente=1),
 Transicion(estado=3, simbolo='0', siguiente=3),
 Transicion(estado=3, simbolo='0', siguiente=4),
 Transicion(estado=3, simbolo='1', siguiente=3),
 Transicion(estado=4, simbolo='0', siguiente=5),
 Transicion(estado=5, simbolo='1', siguiente=6),
 Transicion(estado=6, simbolo='0', siguiente=7),
 Transicion(estado=7, simbolo='0', siguiente=8),
 Transicion(estado=0, simbolo='', siguiente=1),
 Transicion(estado=0, simbolo='', siguiente=3))

In [48]:
NC.estado_inicial

0

In [49]:
NC.estados_de_aceptacion

frozenset({2, 8})

In [50]:
def estrella(NA: af.AFN) -> af.AFN:
    NA = estados_enteros(NA, inicio=1)
    
    programa = list(NA.programa)
    programa.append(af.Transicion(0, '', NA.estado_inicial))
    for estado in NA.estados_de_aceptacion:
        programa.append(af.Transicion(estado, '', NA.estado_inicial))
    
    estado_inicial = 0
    estados_de_aceptacion = list(NA.estados_de_aceptacion) + [0]
    
    return af.AFN(programa, estado_inicial, estados_de_aceptacion)

In [51]:
NC = estrella(NA)
NC.programa

(Transicion(estado=1, simbolo='0', siguiente=2),
 Transicion(estado=1, simbolo='1', siguiente=1),
 Transicion(estado=2, simbolo='1', siguiente=2),
 Transicion(estado=2, simbolo='0', siguiente=1),
 Transicion(estado=2, simbolo='', siguiente=1),
 Transicion(estado=0, simbolo='', siguiente=1))

In [52]:
NC.estado_inicial

0

In [53]:
NC.estados_de_aceptacion

frozenset({0, 2})

## Expresiones regulares en la práctica

In [54]:
import re

$\{h\}\circ\{o\}\circ\{\ell\}\circ\{a\} = \{ho\ell a\}$

In [88]:
patron = re.compile('hola')

In [89]:
bool(patron.fullmatch('hola'))

True

In [90]:
bool(patron.fullmatch('adios'))

False

In [91]:
patron_A = re.compile('cara|cola')

In [92]:
patron_A.fullmatch('cara')

<re.Match object; span=(0, 4), match='cara'>

In [93]:
patron_A.fullmatch('cola')

<re.Match object; span=(0, 4), match='cola'>

In [94]:
patron_A.fullmatch('col')

In [95]:
patron_A.fullmatch('chones')

In [96]:
patron_B = re.compile('col|chones')

In [98]:
patron_B.fullmatch('col')

<re.Match object; span=(0, 3), match='col'>

In [99]:
patron_B.fullmatch('chones')

<re.Match object; span=(0, 6), match='chones'>

In [100]:
patron_B.fullmatch('cara')

In [101]:
patron_B.fullmatch('cola')

In [102]:
patron_B.fullmatch('cola')

In [103]:
patron_AUB = re.compile(patron_A.pattern + '|' + patron_B.pattern)

In [109]:
patron_AUB.fullmatch('cara')

<re.Match object; span=(0, 4), match='cara'>

In [110]:
patron_AUB.fullmatch('chones')

<re.Match object; span=(0, 6), match='chones'>

In [111]:
patron_AB = re.compile('(' + patron_A.pattern + ')(' + patron_B.pattern + ')' )
patron_AB.pattern

'(cara|cola)(col|chones)'

In [112]:
patron_AB.fullmatch('carachones')

<re.Match object; span=(0, 10), match='carachones'>

In [113]:
patron_AB.fullmatch('carachon')

In [114]:
patron_AB.fullmatch('colachones')

<re.Match object; span=(0, 10), match='colachones'>

In [115]:
patron_As = re.compile('(cara|cola)*')

In [116]:
patron_As.fullmatch('caracara')

<re.Match object; span=(0, 8), match='caracara'>

In [118]:
patron_As.fullmatch('caracola')

<re.Match object; span=(0, 8), match='caracola'>

In [119]:
patron_As.fullmatch('caracaracara')

<re.Match object; span=(0, 12), match='caracaracara'>

In [120]:
patron_As.fullmatch('caracolacara')

<re.Match object; span=(0, 12), match='caracolacara'>

In [122]:
r'\d'

'\\d'